<a href="https://colab.research.google.com/github/JCaballerot/Recommender-Systems/blob/main/XGBoost_Recommender/Content_RecSys_BERT_y_XGBoost_Book_Crossing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
  <img src="https://storage.googleapis.com/kaggle-datasets-images/1661575/2726067/684ac0c4c14cb46d1047ccb620b45cac/dataset-cover.jpg?t=2021-10-21-03-18-09" width="800" height="300">
</center>


# **Content RecSys: BERT y XGBoost Book-Crossing**


---
## Tabla de Contenidos

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
1. <a href="#item1">Introducción</a>  
2. <a href="#item4">Descripción del Dataset</a>  
3. <a href="#item4">Preprocesamiento de Datos</a>  
4. <a href="#item4">Muestreo/Enmascaramiento</a>  
5. <a href="#item4">Feature engineering</a>  
6. <a href="#item4">Preparación de los Datos para el Modelo</a>  
7. <a href="#item4">Entrenamiento del Modelo con XGBoost</a>  
8. <a href="#item4">Evaluación del Modelo</a>  
9. <a href="#item4">Generación de Recomendaciones</a>  
10. <a href="#item4">Conclusiones</a>  

</font>
</div>

---

## 1. Introducción

El dataset de **Book-Crossing** es un recurso rico en información sobre las interacciones de usuarios con libros, capturando calificaciones, títulos, autores y otros datos relevantes. Este laboratorio propone desarrollar un sistema de recomendación basado en **machine learning**, combinando técnicas avanzadas de procesamiento de texto con **BERT** y aprendizaje supervisado mediante **XGBoost**.

El objetivo principal es predecir si un usuario disfrutará un libro en particular, basado en sus interacciones pasadas y características tanto del usuario como del libro. Además, se busca generar recomendaciones personalizadas y evaluar métricas como la diversidad global de las recomendaciones.

A lo largo del laboratorio, se explorarán diferentes técnicas de ingeniería de características, como el uso de variables cuantitativas, target encoding para variables categóricas, y **embeddings** textuales obtenidos con **BERT**. Finalmente, se entrenará un modelo de **XGBoost**, y se evaluará su desempeño utilizando métricas como el coeficiente de Gini.

## 2. Descripción del Dataset

El dataset de Book-Crossing contiene 3 archivos principales:

- **BX-Users.csv:** Información sobre los usuarios.
- **BX-Books.csv:** Información sobre los libros.
- **BX-Book-Ratings.csv:** Calificaciones dadas por los usuarios a los libros.

El dataset puede ser descargado desde: Book-Crossing Dataset.



<a name="3.1"></a>

**2.1. BX-Users.csv**

Contiene información de los usuarios:

- **User-ID:** Identificador único del usuario.
- **Location:** Ubicación del usuario.
- **Age:** Edad del usuario.



<a name="3.2"></a>

**2.2. BX-Books.csv**

Contiene información de los libros:

- **SBN:** Identificador único del libro.
- **Book-Title:** Título del libro.
- **Book-Author:** Autor del libro.
- **Year-Of-Publication:** Año de publicación.
- **Publisher:** Editorial.


<a name="3.3"></a>

**2.3. BX-Book-Ratings.csv**
Contiene las calificaciones de los usuarios:

- **User-ID:** Identificador del usuario.
- **ISBN:** Identificador del libro.
- **Book-Rating:** Calificación dada al libro (0-10).


## 4. Preprocesamiento de Datos


En esta sección, cargaremos los datos y realizaremos una limpieza y exploración inicial.



**4.1. Carga de los Datos**

In [6]:

# Crear la carpeta llamada 'dataset'
!mkdir -p dataset

# Descargar los archivos CSV y guardarlos en la carpeta 'dataset'
!wget -O dataset/BX-Book-Ratings.csv https://raw.githubusercontent.com/bigsnarfdude/guide-to-data-mining/master/BX-Dump/BX-Book-Ratings.csv
!wget -O dataset/BX-Books.csv https://raw.githubusercontent.com/bigsnarfdude/guide-to-data-mining/master/BX-Dump/BX-Books.csv
!wget -O dataset/BX-Users.csv https://raw.githubusercontent.com/bigsnarfdude/guide-to-data-mining/master/BX-Dump/BX-Users.csv


--2024-11-26 02:47:34--  https://raw.githubusercontent.com/bigsnarfdude/guide-to-data-mining/master/BX-Dump/BX-Book-Ratings.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29532460 (28M) [text/plain]
Saving to: ‘dataset/BX-Book-Ratings.csv’

dataset/BX-Book-Rat 100%[===================>]  28.16M   182MB/s    in 0.2s    

2024-11-26 02:47:34 (182 MB/s) - ‘dataset/BX-Book-Ratings.csv’ saved [29532460/29532460]

--2024-11-26 02:47:34--  https://raw.githubusercontent.com/bigsnarfdude/guide-to-data-mining/master/BX-Dump/BX-Books.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP 

In [7]:
# Importar las librerías necesarias
import pandas as pd

# Cargar los archivos CSV
ratings = pd.read_csv("dataset/BX-Book-Ratings.csv", sep=";", encoding="ISO-8859-1")
books = pd.read_csv("dataset/BX-Books.csv", sep=";", encoding="ISO-8859-1", on_bad_lines="skip", low_memory=False)
users = pd.read_csv("dataset/BX-Users.csv", sep=";", encoding="ISO-8859-1")

# Asignar manualmente los nombres de las columnas
ratings.columns = ['User-ID', 'ISBN', 'Book-Rating']
books.columns = ['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L']
users.columns = ['User-ID', 'Location', 'Age']


**4.2. Exploración y Limpieza**

Visualizar las primeras filas de cada dataframe:

In [8]:
# Mostrar las primeras filas de ratings
print(ratings.head())

# Mostrar las primeras filas de books
print(books.head())

# Mostrar las primeras filas de users
print(users.head())


   User-ID        ISBN  Book-Rating
0   276726  0155061224            5
1   276727  0446520802            0
2   276729  052165615X            3
3   276729  0521795028            6
4   276733  2080674722            0
         ISBN                                         Book-Title  \
0  0002005018                                       Clara Callan   
1  0060973129                               Decision in Normandy   
2  0374157065  Flu: The Story of the Great Influenza Pandemic...   
3  0393045218                             The Mummies of Urumchi   
4  0399135782                             The Kitchen God's Wife   

            Book-Author Year-Of-Publication                   Publisher  \
0  Richard Bruce Wright                2001       HarperFlamingo Canada   
1          Carlo D'Este                1991             HarperPerennial   
2      Gina Bari Kolata                1999        Farrar Straus Giroux   
3       E. J. W. Barber                1999  W. W. Norton &amp; Company   


**Limpieza de Datos**

Conversión de tipos de datos.

Manejo de valores faltantes.

In [9]:
# Convertir 'Year-Of-Publication' a numérico y manejar errores
books['Year-Of-Publication'] = pd.to_numeric(books['Year-Of-Publication'], errors='coerce')
books['Year-Of-Publication'].fillna(books['Year-Of-Publication'].median(), inplace=True)
books['Year-Of-Publication'] = books['Year-Of-Publication'].astype(int)


<ipython-input-9-7f58c4612ec2>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  books['Year-Of-Publication'].fillna(books['Year-Of-Publication'].median(), inplace=True)


In [10]:
# Manejar valores faltantes en 'Book-Author' y 'Publisher'
books['Book-Author'].fillna('Unknown', inplace=True)
books['Publisher'].fillna('Unknown', inplace=True)


<ipython-input-10-71b7ce4f9bdd>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  books['Book-Author'].fillna('Unknown', inplace=True)
<ipython-input-10-71b7ce4f9bdd>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try 

In [11]:
# Convertir 'Age' a numérico y manejar errores
users['Age'] = pd.to_numeric(users['Age'], errors='coerce')
users['Age'].fillna(users['Age'].median(), inplace=True)
users['Age'] = users['Age'].astype(int)

<ipython-input-11-9482be963f6e>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  users['Age'].fillna(users['Age'].median(), inplace=True)


## 5. División de los Datos

Para evaluar el modelo de manera adecuada, separaremos un 10% de las calificaciones de cada usuario para utilizarlas como conjunto de prueba.

In [21]:
from sklearn.model_selection import GroupShuffleSplit

# Unir ratings con usuarios y libros
data = ratings.merge(users, on='User-ID').merge(books, on='ISBN')[:10000]

# Binarizar el target: 1 si Book-Rating > 5, 0 si <= 5
data['target'] = (data['Book-Rating'] > 7).astype(int)


In [22]:
# Separar en entrenamiento y prueba por usuario
gss = GroupShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
train_idx, test_idx = next(gss.split(data, groups=data['User-ID']))

train_data = data.iloc[train_idx].reset_index(drop=True)
test_data = data.iloc[test_idx].reset_index(drop=True)


## 6. Ingeniería de Características

En esta sección, crearemos nuevas características que ayuden al modelo a predecir con mayor precisión.

**6.1. Variables Cuantitativas**

- Edad del usuario (Age).

- Antigüedad del libro: Año actual menos el año de publicación.

In [23]:
from datetime import datetime

current_year = datetime.now().year

# Calcular la antigüedad del libro
train_data['Book-Age'] = current_year - train_data['Year-Of-Publication']
test_data['Book-Age'] = current_year - test_data['Year-Of-Publication']


**6.2. Target Encoding**

Aplicaremos target encoding a las variables categóricas:

- Location
- Book-Author
- Publisher

In [16]:
%%capture
!pip3 install category_encoders

In [24]:
from category_encoders import TargetEncoder

categorical_features = ['Location', 'Book-Author', 'Publisher']

encoder = TargetEncoder(cols=categorical_features)

# Ajustar el encoder en los datos de entrenamiento
encoder.fit(train_data[categorical_features], train_data['target'])

# Transformar las variables categóricas
train_encoded = encoder.transform(train_data[categorical_features])
test_encoded = encoder.transform(test_data[categorical_features])

# Añadir las variables codificadas al dataframe
train_data = pd.concat([train_data, train_encoded], axis=1)
test_data = pd.concat([test_data, test_encoded], axis=1)


**6.3. Generación de Embeddings con BERT**

Utilizaremos BERT para convertir el título del libro en embeddings numéricos que capturen su semántica.



**Instalación y Carga de BERT**

In [18]:
# Instalar la biblioteca transformers
!pip install transformers

from transformers import BertTokenizer, BertModel
import torch

# Cargar el tokenizador y el modelo preentrenado
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Función para Obtener Embeddings

In [25]:
def get_bert_embedding(text):
    # Tokenización y codificación
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=20)
    # Obtener las representaciones del modelo
    with torch.no_grad():
        outputs = model(**inputs)
    # Usar el embedding del token [CLS] como representación
    embedding = outputs.last_hidden_state[:,0,:].numpy()
    return embedding.flatten()


**Aplicar la Función a los Datos**

Debido a limitaciones computacionales, podemos trabajar con una muestra.

In [26]:
import numpy as np

# Obtener embeddings para el título del libro en el conjunto de entrenamiento
train_embeddings = np.vstack(train_data['Book-Title'].apply(get_bert_embedding))

# Obtener embeddings para el conjunto de prueba
test_embeddings = np.vstack(test_data['Book-Title'].apply(get_bert_embedding))


KeyboardInterrupt: 

**6.4. One-Hot Encoding basado en Productos Calificados**

Crearemos características que indiquen si el usuario ya ha calificado el libro y el puntaje que le dio previamente.

**Crear un Diccionario de Calificaciones por Usuario**

In [27]:
# Diccionario de libros calificados por usuario
user_rated_books = train_data.groupby('User-ID')['ISBN'].apply(set).to_dict()

# Función para verificar si el usuario ya calificó el libro
def has_rated(user_id, isbn):
    return int(isbn in user_rated_books.get(user_id, set()))


**Aplicar la Función a los Datos**

In [28]:
# Añadir el flag de 'ya lo calificó'
train_data['Already_Rated'] = train_data.apply(lambda x: has_rated(x['User-ID'], x['ISBN']), axis=1)
test_data['Already_Rated'] = test_data.apply(lambda x: has_rated(x['User-ID'], x['ISBN']), axis=1)


**Añadir el Puntaje Dado Anteriormente**

In [29]:
# Diccionario de calificaciones previas
user_book_rating = train_data.set_index(['User-ID', 'ISBN'])['Book-Rating'].to_dict()

# Función para obtener el puntaje anterior
def previous_rating(user_id, isbn):
    return user_book_rating.get((user_id, isbn), 0)  # 0 si no existe

# Añadir la calificación previa
train_data['Previous_Rating'] = train_data.apply(lambda x: previous_rating(x['User-ID'], x['ISBN']), axis=1)
test_data['Previous_Rating'] = test_data.apply(lambda x: previous_rating(x['User-ID'], x['ISBN']), axis=1)


In [43]:
train_data.describe()

,User-ID,Book-Rating,Age,Year-Of-Publication,target,Book-Age,Location,Book-Author,Publisher,Already_Rated,Previous_Rating
count,8842.000000,8842.000000,8842.000000,8842.000000,8842.000000,8842.000000,8842.000000,8842.000000,8842.000000,8842.0,8842.000000
mean,241209.162520,2.011536,33.285230,1965.995363,0.144651,58.004637,0.115062,0.146875,0.135391,1.0,2.011536
std,94255.726797,3.443943,8.342845,234.594450,0.351768,234.594450,0.148001,0.052188,0.073314,0.0,3.443943
min,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000265,0.000080,0.008636,1.0,0.000000
25%,277439.000000,0.000000,32.000000,1990.000000,0.000000,25.000000,0.005756,0.124132,0.099395,1.0,0.000000
50%,278418.000000,0.000000,32.000000,1995.000000,0.000000,29.000000,0.066667,0.125830,0.125830,1.0,0.000000
75%,278418.000000,5.000000,32.000000,1999.000000,0.000000,34.000000,0.173795,0.162347,0.179571,1.0,5.000000
max,278854.000000,10.000000,104.000000,2024.000000,1.000000,2024.000000,0.733836,0.383995,0.355733,1.0,10.000000


## 7. Preparación de los Datos para el Modelo

Seleccionaremos las características y prepararemos los conjuntos de entrenamiento y prueba.

['Age',
 'Book-Age',
 'Already_Rated',
 'Previous_Rating',
 'Location',
 'Book-Author',
 'Publisher']

In [48]:
# Variables cuantitativas
quant_features = ['Age', 'Book-Age', 'Already_Rated', 'Previous_Rating']

# Variables de target encoding
encoded_features = encoder.get_feature_names()

# Combinar todas las características
X_train = np.hstack([train_data[quant_features].values])
X_test = np.hstack([test_data[quant_features].values])

# Variables objetivo
y_train = train_data['target'].values
y_test = test_data['target'].values


/usr/local/lib/python3.10/dist-packages/category_encoders/utils.py:379: FutureWarning: `get_feature_names` is deprecated in all of sklearn. Use `get_feature_names_out` instead.
  warnings.warn("`get_feature_names` is deprecated in all of sklearn. Use `get_feature_names_out` instead.",


## 8. Entrenamiento del Modelo con XGBoost

Entrenaremos un modelo de clasificación utilizando XGBoost.

In [52]:
# Instalar XGBoost
!pip install xgboost

import xgboost as xgb

# Crear el modelo XGBoost
model = xgb.XGBClassifier(n_estimators = 100,
                          use_label_encoder=False,
                          eval_metric='logloss',
                          max_depth = 8,
                          learning_rate  = 0.1,
                          min_child_weight = 100,
                          subsample = 0.6,
                          colsample_bytree = 0.6)

# Entrenar el modelo
model.fit(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [03:08:59] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
              max_leaves=None, min_child_weight=100, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [47]:
X_train

array([[32, 23, 1, ..., 0.12583027157766635, 'Heinle',
        0.12583027157766635],
       [16, 28, 1, ..., 0.2419809221366398, 'Warner Books',
        0.18784529946905126],
       [16, 25, 1, ..., 0.12583027157766635,
        'Cambridge University Press', 0.12230704885299244],
       ...,
       [32, 25, 1, ..., 0.2767723139365271, 'Buy Books on the web.com',
        0.25593874594066424],
       [32, 25, 1, ..., 0.2767723139365271, 'Infinity Publishing (PA)',
        0.26598276451508585],
       [32, 24, 1, ..., 0.2767723139365271, 'Infinity Publishing (PA)',
        0.26598276451508585]], dtype=object)

## 9. Evaluación del Modelo

Evaluaremos el rendimiento del modelo utilizando el coeficiente de Gini.

**Función para Calcular el Gini**

In [ ]:
from sklearn.metrics import roc_auc_score

def gini_coefficient(y_true, y_score):
    # Calcular el AUC-ROC
    auc = roc_auc_score(y_true, y_score)
    return 2 * auc - 1


Evaluar el Modelo

In [ ]:
# Predecir probabilidades en el conjunto de prueba
y_probs = model.predict_proba(X_test)[:,1]

# Calcular el Gini
gini = gini_coefficient(y_test, y_probs)
print(f'Coeficiente de Gini en el conjunto de prueba: {gini:.4f}')


## 10. Generación de Recomendaciones

Generaremos recomendaciones personalizadas para los usuarios y calcularemos la diversidad global.

**10.1. Predicción para Libros No Calificados**

Para cada usuario, recomendamos los 10 libros con mayor probabilidad de gustarle.

In [ ]:
from tqdm import tqdm

# Obtener todos los ISBN únicos
all_isbns = books['ISBN'].unique()

# Diccionario de libros ya calificados por usuario
user_rated_books_all = data.groupby('User-ID')['ISBN'].apply(set).to_dict()

recommendations = {}

for user_id in tqdm(test_data['User-ID'].unique()):
    # Libros no calificados por el usuario
    unrated_books = set(all_isbns) - user_rated_books_all.get(user_id, set())
    # Crear un dataframe temporal
    temp_df = pd.DataFrame({'User-ID': user_id, 'ISBN': list(unrated_books)})
    # Unir con información de usuarios y libros
    temp_df = temp_df.merge(users, on='User-ID').merge(books, on='ISBN', how='left')
    # Manejar valores faltantes
    temp_df['Age'].fillna(users['Age'].median(), inplace=True)
    temp_df['Year-Of-Publication'].fillna(books['Year-Of-Publication'].median(), inplace=True)
    temp_df['Book-Author'].fillna('Unknown', inplace=True)
    temp_df['Publisher'].fillna('Unknown', inplace=True)
    temp_df['Book-Title'].fillna('', inplace=True)
    # Calcular 'Book-Age'
    temp_df['Year-Of-Publication'] = temp_df['Year-Of-Publication'].astype(int)
    temp_df['Book-Age'] = current_year - temp_df['Year-Of-Publication']
    # Target Encoding
    temp_encoded = encoder.transform(temp_df[categorical_features])
    temp_df = pd.concat([temp_df, temp_encoded], axis=1)
    # Embeddings con BERT
    temp_embeddings = np.vstack(temp_df['Book-Title'].apply(get_bert_embedding))
    # Variables cuantitativas
    temp_df['Already_Rated'] = 0  # No lo han calificado aún
    temp_df['Previous_Rating'] = 0  # No hay calificación previa
    temp_quant_features = temp_df[quant_features + encoded_features].values
    X_temp = np.hstack([temp_quant_features, temp_embeddings])
    # Predecir probabilidades
    temp_probs = model.predict_proba(X_temp)[:,1]
    temp_df['probability'] = temp_probs
    # Obtener los 10 libros con mayor probabilidad
    top_10 = temp_df.nlargest(10, 'probability')['ISBN'].tolist()
    recommendations[user_id] = top_10


**10.2. Cálculo de la Diversidad Global**

Calculamos la proporción de libros únicos recomendados respecto al total del catálogo.

In [ ]:
# Conjunto de libros recomendados
recommended_books = set()

for recs in recommendations.values():
    recommended_books.update(recs)

diversity = len(recommended_books) / len(all_isbns)
print(f'Diversidad Global de Recomendaciones: {diversity:.4f}')


## 11. Conclusiones

En este laboratorio, hemos implementado un sistema de recomendación que combina técnicas avanzadas de procesamiento de lenguaje natural y aprendizaje automático:

**BERT** nos permitió convertir títulos de libros en representaciones numéricas que capturan su significado semántico.

**XGBoost** fue utilizado para entrenar un modelo de clasificación capaz de predecir si a un usuario le gustará un libro.

Mediante **feature engineerering**, incorporamos información relevante sobre usuarios y libros, mejorando la capacidad predictiva del modelo.

Las recomendaciones generadas ofrecen una diversidad global significativa, lo cual es beneficioso para mantener el interés del usuario y promover una exploración más amplia del catálogo.

Este enfoque demuestra cómo la integración de diferentes técnicas puede conducir a soluciones efectivas en el desarrollo de sistemas de recomendación personalizados.



---
## Gracias por completar este laboratorio!